In [2]:
import numpy as np
import qubex as qx

In [3]:
flattop = qx.pulse.FlatTop(
    duration=30,
    amplitude=0.5,
    tau=10,
)

# Plot the pulse
flattop.plot()

# Print the pulse values
print(flattop.values)

[0.01223587+0.j 0.10305369+0.j 0.25      +0.j 0.39694631+0.j
 0.48776413+0.j 0.5       +0.j 0.5       +0.j 0.5       +0.j
 0.5       +0.j 0.5       +0.j 0.48776413+0.j 0.39694631+0.j
 0.25      +0.j 0.10305369+0.j 0.01223587+0.j]


In [4]:
# Create a Gaussian pulse
gaussian = qx.pulse.Gaussian(
    duration=60,
    amplitude=0.5,
    sigma=10,
)

# Create a pulse array
arr = qx.PulseArray(
    [
        gaussian.shifted(np.pi / 2),
        qx.VirtualZ(np.pi / 2),
        gaussian.shifted(-np.pi / 2),
    ]
)

# Plot the logical pulse array
arr.plot(show_physical_pulse=False)

# Plot the physical pulse array
arr.plot(show_physical_pulse=True)

# Print the pulse array values
print(arr.values)

[ 1.18013407e-19+1.92730519e-03j  4.64785903e-19+7.59052983e-03j
  1.01635640e-18+1.65983597e-02j  1.85439755e-18+3.02846102e-02j
  3.06943221e-18+5.01276321e-02j  4.74821085e-18+7.75441679e-02j
  6.95478774e-18+1.13580303e-01j  9.70733899e-18+1.58532877e-01j
  1.29552055e-17+2.11574562e-01j  1.65625863e-17+2.70487560e-01j
  2.03057368e-17+3.31617849e-01j  2.38886795e-17+3.90131743e-01j
  2.69782617e-17+4.40588449e-01j  2.92538474e-17+4.77751583e-01j
  3.04617558e-17+4.97478225e-01j  3.04617558e-17+4.97478225e-01j
  2.92538474e-17+4.77751583e-01j  2.69782617e-17+4.40588449e-01j
  2.38886795e-17+3.90131743e-01j  2.03057368e-17+3.31617849e-01j
  1.65625863e-17+2.70487560e-01j  1.29552055e-17+2.11574562e-01j
  9.70733899e-18+1.58532877e-01j  6.95478774e-18+1.13580303e-01j
  4.74821085e-18+7.75441679e-02j  3.06943221e-18+5.01276321e-02j
  1.85439755e-18+3.02846102e-02j  1.01635640e-18+1.65983597e-02j
  4.64785903e-19+7.59052983e-03j  1.18013407e-19+1.92730519e-03j
 -1.92730519e-03-2.360268

In [5]:
# Create an arbitrary pulse
iq_array = [
    0.1 + 0.2j,
    0.2 + 0.3j,
    0.3 + 0.4j,
    0.4 + 0.6j,
    0.5 + 0.6j,
]
arbit = qx.Pulse(iq_array)

# Plot the pulse
arbit.plot()

# Plot the reverse of the pulse
arbit.inverted().plot()

In [6]:
# Create a pulse schedule for the channels
with qx.PulseSchedule() as ps1:
    ps1.add("Q00", flattop.repeated(5))
    ps1.add("Q01", arr.scaled(2))
    ps1.barrier()
    ps1.add("Q00", arbit.repeated(3))
    ps1.barrier(["Q00", "Q01"])
    ps1.add("Q01", qx.pulse.FlatTop(duration=100, amplitude=1, tau=10))
    ps1.add("Q02", qx.PulseArray([gaussian, qx.VirtualZ(np.pi / 2), gaussian]))

# Plot the pulse schedule
ps1.plot()

# Get the PulseArray dictionary of the channels
ps1.get_sequences()

{'Q00': PulseArray([FlatTop(75), Pulse(15), Blank(50)]),
 'Q01': PulseArray([PulseArray(60), Blank(15), Blank(15), FlatTop(50)]),
 'Q02': PulseArray([Blank(75), PulseArray(60), Blank(5)])}

In [7]:
targets = ["Q00", "Q01", "Q02"]

hpi = qx.pulse.FlatTop(
    duration=30,
    amplitude=0.5,
    tau=10,
)
def ltmeas(time: int):
	with qx.PulseSchedule(targets) as ps:
		for target in targets:
			ps.add(target, hpi)
			ps.add(target, hpi)
			ps.add(target, qx.pulse.Blank(time))  # 指定した待ち時間だけ待機
	return ps


ltmeas(50).plot()